In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv("C:/Training/Academy/Statistics (Python)/Cases/Vehicle Silhouettes/Vehicle.csv")
df.head()

Comp  Circ  D.Circ  Rad.Ra  Pr.Axis.Ra  Max.L.Ra  Scat.Ra  Elong  \
0    95    48      83     178          72        10      162     42   
1    91    41      84     141          57         9      149     45   
2   104    50     106     209          66        10      207     32   
3    93    41      82     159          63         9      144     46   
4    85    44      70     205         103        52      149     45   

   Pr.Axis.Rect  Max.L.Rect  Sc.Var.Maxis  Sc.Var.maxis  Ra.Gyr  Skew.Maxis  \
0            20         159           176           379     184          70   
1            19         143           170           330     158          72   
2            23         158           223           635     220          73   
3            19         143           160           309     127          63   
4            19         144           241           325     188         127   

   Skew.maxis  Kurt.maxis  Kurt.Maxis  Holl.Ra Class  
0           6          16         187      197   van  
1           9          14         189      199   van  
2          14           9         188      196  saab  
3           6          10         199      207   van  
4           9          11         180      183   bus

In [3]:
df['Class'].unique()

array(['van', 'saab', 'bus', 'opel'], dtype=object)

In [4]:
dum_df = pd.get_dummies(df)
dum_df.head()

Comp  Circ  D.Circ  Rad.Ra  Pr.Axis.Ra  Max.L.Ra  Scat.Ra  Elong  \
0    95    48      83     178          72        10      162     42   
1    91    41      84     141          57         9      149     45   
2   104    50     106     209          66        10      207     32   
3    93    41      82     159          63         9      144     46   
4    85    44      70     205         103        52      149     45   

   Pr.Axis.Rect  Max.L.Rect  ...  Ra.Gyr  Skew.Maxis  Skew.maxis  Kurt.maxis  \
0            20         159  ...     184          70           6          16   
1            19         143  ...     158          72           9          14   
2            23         158  ...     220          73          14           9   
3            19         143  ...     127          63           6          10   
4            19         144  ...     188         127           9          11   

   Kurt.Maxis  Holl.Ra  Class_bus  Class_opel  Class_saab  Class_van  
0         187      197      False       False       False       True  
1         189      199      False       False       False       True  
2         188      196      False       False        True      False  
3         199      207      False       False       False       True  
4         180      183       True       False       False      False  

[5 rows x 22 columns]

In [5]:
X = dum_df.iloc[:,:-4]
X.head()

Comp  Circ  D.Circ  Rad.Ra  Pr.Axis.Ra  Max.L.Ra  Scat.Ra  Elong  \
0    95    48      83     178          72        10      162     42   
1    91    41      84     141          57         9      149     45   
2   104    50     106     209          66        10      207     32   
3    93    41      82     159          63         9      144     46   
4    85    44      70     205         103        52      149     45   

   Pr.Axis.Rect  Max.L.Rect  Sc.Var.Maxis  Sc.Var.maxis  Ra.Gyr  Skew.Maxis  \
0            20         159           176           379     184          70   
1            19         143           170           330     158          72   
2            23         158           223           635     220          73   
3            19         143           160           309     127          63   
4            19         144           241           325     188         127   

   Skew.maxis  Kurt.maxis  Kurt.Maxis  Holl.Ra  
0           6          16         187      197  
1           9          14         189      199  
2          14           9         188      196  
3           6          10         199      207  
4           9          11         180      183

In [6]:
y = dum_df.iloc[:,-4:]
y.head()

Class_bus  Class_opel  Class_saab  Class_van
0      False       False       False       True
1      False       False       False       True
2      False       False        True      False
3      False       False       False       True
4       True       False       False      False

In [7]:
le = LabelEncoder()
le_y = le.fit_transform(df['Class'])

In [8]:
y = y.values  # converting to numpy

In [9]:
scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=23,stratify=le_y)
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 


In [10]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([592, 18])
torch.Size([592, 4])


In [11]:
type(X_torch)

torch.Tensor

In [12]:
X_scl_trn.shape[1]

18

In [13]:
torch.manual_seed(2022)
# Create a model
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1], out_features=10),
                      nn.ReLU(),
                      nn.Linear(10, 5),
                      nn.ReLU(),
                      nn.Linear(5,4))

In [14]:
print(model)

Sequential(
  (0): Linear(in_features=18, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=5, bias=True)
  (3): ReLU()
  (4): Linear(in_features=5, out_features=4, bias=True)
)


In [15]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)

In [16]:
y_torch.size()

torch.Size([592, 4])

In [17]:
y_pred = model(X_torch.float())
print(y_torch.shape)
print(y_pred.shape)

torch.Size([592, 4])
torch.Size([592, 4])


In [18]:
y_pred[:5]

tensor([[-0.0898, -0.0777, -0.3168,  0.3049],
        [-0.0288, -0.0246, -0.3460,  0.2480],
        [-0.0389, -0.0450, -0.3326,  0.2538],
        [-0.0603, -0.0459, -0.3355,  0.2793],
        [-0.0417, -0.0362, -0.3398,  0.2605]], grad_fn=<SliceBackward0>)

In [19]:
y_torch.float().size()

torch.Size([592, 4])

### Initial Log Loss

In [20]:
loss = criterion(y_pred, y_torch.float())
loss

tensor(1.4133, grad_fn=<DivBackward1>)

In [21]:
X_torch_test = torch.from_numpy(X_scl_tst)
type(X_torch_test)

torch.Tensor

In [22]:
y_torch_test = torch.from_numpy(y_test)
y_torch_test.size()

torch.Size([254, 4])

### Training Loop

In [23]:
for epoch in np.arange(0,1000):
       # Forward pass: Compute predicted y by passing x to the model
       y_pred_prob = model(X_torch.float())
       y_pred_prob_test = model(X_torch_test.float())
        
       # Compute and print loss
       loss = criterion(y_pred_prob, y_torch.float())
       tst_loss = criterion(y_pred_prob_test, y_torch_test.float() )
       if epoch%100 == 0:
          print('epoch: ', epoch+1,' train loss: ', loss.item(), " test loss:", tst_loss.item())

       # Zero gradients, perform a backward pass, and update the weights.
       optimizer.zero_grad()

       # perform a backward pass (backpropagation)
       loss.backward()

       # Update the parameters
       optimizer.step()
#print('epoch: ', epoch+1,' train loss: ', loss.item(), " test loss:", tst_loss.item())

epoch:  1  train loss:  1.4132686853408813  test loss: 1.4139108657836914
epoch:  101  train loss:  0.6237322092056274  test loss: 0.6221696138381958
epoch:  201  train loss:  0.3950933516025543  test loss: 0.46601688861846924
epoch:  301  train loss:  0.3315504789352417  test loss: 0.4830019176006317
epoch:  401  train loss:  0.3071947991847992  test loss: 0.5041605234146118
epoch:  501  train loss:  0.286405086517334  test loss: 0.5081126689910889
epoch:  601  train loss:  0.2629603147506714  test loss: 0.5109526515007019
epoch:  701  train loss:  0.23353512585163116  test loss: 0.46336206793785095
epoch:  801  train loss:  0.21399101614952087  test loss: 0.4483228027820587
epoch:  901  train loss:  0.2002653181552887  test loss: 0.45071542263031006


### Training Set Log Loss after training loop execution

In [24]:
loss

tensor(0.1884, grad_fn=<DivBackward1>)

In [25]:
#torch.set_printoptions(precision=3, sci_mode=False)

In [26]:
y_wt_sum = model(X_torch_test.float()) 
softmax = nn.Softmax(dim=1)
pred_proba = softmax(y_wt_sum)
pred_proba

tensor([[5.4205e-04, 4.9584e-07, 2.4284e-09, 9.9946e-01],
        [3.0046e-12, 5.0490e-01, 4.9510e-01, 7.1269e-23],
        [9.9329e-01, 3.5821e-03, 3.1279e-03, 1.1714e-08],
        ...,
        [1.6399e-08, 7.4719e-01, 2.5281e-01, 1.8404e-21],
        [1.0000e+00, 2.0685e-08, 1.4334e-11, 1.3498e-10],
        [9.9994e-01, 2.9080e-05, 1.8560e-05, 8.9138e-06]],
       grad_fn=<SoftmaxBackward0>)

In [27]:
pred_proba.size()

torch.Size([254, 4])

In [28]:
y_pred = np.argmax(pred_proba.detach().numpy(), axis=1 )

y_pred[:5]

array([3, 1, 0, 0, 2], dtype=int64)

In [29]:
y_test_lbl = np.argmax(y_test, axis=1)
y_test_lbl[:5]

array([3, 1, 0, 0, 2], dtype=int64)

### Test Set Accuracy Score

In [30]:
print(accuracy_score(y_test_lbl,y_pred))

0.8346456692913385
